In [1]:
################### 데이터 불러오기
import numpy
import pandas as pd

"""
The y=target is to maximize this equation ASAP:
    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
    What are the best values for the 6 weights w1 to w6?
    We are going to use the genetic algorithm for the best possible values after a number of generations.
"""
# data
restaurants = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/restaurants.txt", sep = '\t')
orders = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/orders.txt", sep = '\t')
couriers = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/couriers.txt", sep = '\t')
rest_orders = pd.merge(restaurants, orders, how='inner', on=['restaurant'])
rest_orders.columns = ['restaurant', 'rx', 'ry', 'order', 'ox', 'oy', 'placement_time', 'ready_time']

# 주문 건
order = rest_orders.sample(n=1).values
order = ['r1',8708,5633,'o1',9131,7497,743,753]
print("order", order)
# 근무중인 라이더
rider = couriers[(couriers['on_time'] <= order[6]) & (couriers['off_time'] >= order[7])]
data_len = len(rider)
print("근무중인 라이더", len(rider), "명")

order ['r1', 8708, 5633, 'o1', 9131, 7497, 743, 753]
근무중인 라이더 9 명


In [11]:
"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""
# Defining the population size.
num_generations = 50

sol_per_pop = 8
num_parents_mating = 6
# weights = [0.1,0.3,0.5,0.7]

numpy.random.seed(0)
# Inputs of the equation. # 내가 업데이트 해야하는 값
delivered_num = numpy.random.uniform(low=3, high=25, size=data_len).astype(int)
dist_x = numpy.array(rider['x'])
dist_y = numpy.array(rider['y'])
on_time = numpy.array(rider['on_time'])
off_time = numpy.array(rider['off_time'])
speed = numpy.random.uniform(low=30, high=60, size=data_len).astype(int)

new_population = numpy.stack((delivered_num, dist_x, dist_y, on_time, off_time, speed), axis=0)
# equation_inputs = data[0]
print("new_population",new_population)

# initialize population
'''new_delivered_num = numpy.random.uniform(low=3, high=25, size=sol_per_pop).astype(int)
new_dist_x = numpy.random.uniform(low=0, high=12555, size=sol_per_pop).astype(int)
new_dist_y = numpy.random.uniform(low=0, high=11777, size=sol_per_pop).astype(int)
new_on_time = numpy.full(sol_per_pop, numpy.unique(on_time)[0])
new_off_time = numpy.full(sol_per_pop, numpy.unique(off_time)[0])
new_speed = numpy.random.uniform(low=30, high=60, size=sol_per_pop).astype(int)

new_population = numpy.stack((new_delivered_num, new_dist_x, new_dist_y, new_on_time, new_off_time, new_speed), axis=1)
print("new_population",new_population)'''

# Number of the weights we are looking to optimize.
num_weights = len(new_population[0])

# The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
pop_size = (sol_per_pop,num_weights)
print("pop_size",pop_size)

new_population [[   15    18    16    14    12    17    12    22    24]
 [ 3810  3217 11678  7651 10855  7038  5013  8236  4981]
 [ 7415  9891  3104  4332  7784  5694  2778  1518  4827]
 [  690   690   690   690   690   690   690   690   690]
 [  810   840   810   810   810   840   810   810   840]
 [   41    53    45    47    57    32    32    30    54]]
pop_size (8, 9)


In [38]:
min(couriers['x']),max(couriers['x'])

(0, 12555)

In [39]:
min(couriers['y']),max(couriers['y'])

(0, 11777)

In [10]:
delivered_num`

array([15, 18, 16, 14, 12, 17, 12, 22, 24])

In [3]:
############## stack = 1
import numpy

def standard_deviation(pop):
    return numpy.std(pop[:,0])

def calculate_speed(pop):
    speed = pop[:,5]
    return speed

def calculate_trust(pop):
    # 배달원의 총 배달 시간 / 배달 건 수
    trust = pop[:,4]-pop[:,3] / pop[:,0]
    return trust
    
# Calculating the fitness value of each solution in the current population.
# The fitness function calulates the sum of products between each input and its corresponding weight.

def cal_pop_fitness(pop):
    std = standard_deviation(pop)
    speed = calculate_speed(pop)
    trust = calculate_trust(pop)
    
    log_std = numpy.log1p(std)
    log_speed = numpy.log1p(speed)
    log_trust = numpy.log1p(trust)
    return 1/log_std + log_speed +log_trust

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover, num_mutations=1):
    mutations_counter = numpy.uint8(offspring_crossover.shape[1] / num_mutations)
    # Mutation changes a number of genes as defined by the num_mutations argument. The changes are random.
    for idx in range(offspring_crossover.shape[0]):
        gene_idx = mutations_counter - 1
        for mutation_num in range(num_mutations):
            # The random value to be added to the gene.
            random_value = numpy.random.uniform(-1.0, 1.0, 1)
            offspring_crossover[idx, gene_idx] = offspring_crossover[idx, gene_idx] + random_value
            gene_idx = gene_idx + mutations_counter
    return offspring_crossover

print(cal_pop_fitness(new_population))

[11.00550848 11.30488659 11.10023138 11.13473557 11.31313505 10.80959689
 10.7491996  10.72077784 11.33510438]


In [4]:
########### Genetic Algoritm ###########

best_outputs = []
# num_generations = 500
for generation in range(num_generations):
    print("Generation : ", generation)
    # Measuring the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population)
    print("Fitness")
    print(fitness)

    best_outputs.append(numpy.max(fitness))
    # The best result in the current iteration.
    print("Best result : ", numpy.max(fitness))
    
    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, 
                                      num_parents_mating)
    print("Parents")
    print(parents)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, offspring_size=(sol_per_pop-parents.shape[0], num_weights))
                                    # offspring_size=(pop_size[0]-parents.shape[0], num_weights)) 
    print("Crossover")
    print(offspring_crossover)

    # Adding some variations to the offspring using mutation.
    offspring_mutation = mutation(offspring_crossover, num_mutations=2)
    print("Mutation")
    print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation
    
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(new_population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])


import matplotlib.pyplot
matplotlib.pyplot.plot(best_outputs)
matplotlib.pyplot.xlabel("Iteration")
matplotlib.pyplot.ylabel("Fitness")
matplotlib.pyplot.show()

Generation :  0
Fitness
[11.00550848 11.30488659 11.10023138 11.13473557 11.31313505 10.80959689
 10.7491996  10.72077784 11.33510438]
Best result :  11.335104384746167
Parents
[[   24.  4981.  4827.   690.   840.    54.]
 [   12. 10855.  7784.   690.   810.    57.]
 [   18.  3217.  9891.   690.   840.    53.]
 [   14.  7651.  4332.   690.   810.    47.]
 [   16. 11678.  3104.   690.   810.    45.]
 [   15.  3810.  7415.   690.   810.    41.]]
Crossover
[[   24.  4981.  4827.   690.   810.    57.]
 [   12. 10855.  7784.   690.   840.    53.]]
Mutation
[[   24.          4981.          4827.5563135    690.
    810.            57.7400243 ]
 [   12.         10855.          7784.95723668   690.
    840.            53.59831713]]


ValueError: could not broadcast input array from shape (2,6) into shape (3,6)

In [32]:
function_inputs = numpy.array([[15, 18, 16, 14, 12, 17, 12, 22, 24],
                   [3810, 3217, 11678, 7651, 10855, 7038, 5013, 8236, 4981],
                   [7415, 9891, 3104, 4332, 7784, 5694, 2778, 1518, 4827],
                   [690, 690, 690, 690, 690, 690, 690, 690, 690],
                   [810, 840, 810, 810, 810, 840, 810, 810, 840],
                   [41, 53,  45, 47, 57, 32, 32, 30, 54]]) # Function inputs.
# desired_output = 44 # Function output.

# def fitness_func(solution, solution_idx):
#     solution = solution.reshape(len(function_inputs),len(function_inputs[0]))
   
std = numpy.std(function_inputs[0])
log_std = numpy.log1p(std)

speed = function_inputs[5]
sc_speed = ((speed - min(speed)) / (max(speed) - min(speed)))

trust = (function_inputs[4]- function_inputs[3]) / function_inputs[0]
sc_trust = ((trust - min(trust)) / (max(trust) - min(trust)))

In [33]:
log_std

1.5924504473118544

In [34]:
sc_speed

array([0.40740741, 0.85185185, 0.55555556, 0.62962963, 1.        ,
       0.07407407, 0.07407407, 0.        , 0.88888889])

In [35]:
sc_trust

array([0.56      , 0.63333333, 0.45      , 0.68571429, 1.        ,
       0.74117647, 1.        , 0.        , 0.175     ])

In [36]:
1/log_std + sc_speed + sc_trust

array([1.59537043, 2.11314821, 1.63351858, 1.94330694, 2.62796302,
       1.44321357, 1.7020371 , 0.62796302, 1.69185191])

In [21]:
function_inputs = numpy.array([[15, 18, 16, 14, 12, 17, 12, 22, 24],
                   [3810, 3217, 11678, 7651, 10855, 7038, 5013, 8236, 4981],
                   [7415, 9891, 3104, 4332, 7784, 5694, 2778, 1518, 4827],
                   [690, 690, 690, 690, 690, 690, 690, 690, 690],
                   [810, 840, 810, 810, 810, 840, 810, 810, 840],
                   [41, 53,  45, 47, 57, 32, 32, 30, 54]])

In [22]:
sum_fit = 0 
std = numpy.std(function_inputs[0])
log_std = numpy.log1p(std)

speed = function_inputs[5]
sc_speed = ((speed - min(speed)) / (max(speed) - min(speed)))

trust = (function_inputs[4]- function_inputs[3]) / function_inputs[0]
sc_trust = ((trust - min(trust)) / (max(trust) - min(trust)))

# Calculating the fitness value of each solution in the current population.
# The fitness function calulates the sum of products between each input and its corresponding weight.
#     output = numpy.sum(solution*function_inputs)
#     fitness = 1.0 / numpy.abs(output - desired_output)
sum_fit += 1/log_std + sc_speed + sc_trust
sum_fit

array([1.59537043, 2.11314821, 1.63351858, 1.94330694, 2.62796302,
       1.44321357, 1.7020371 , 0.62796302, 1.69185191])